# End-To-End Python Client Example

This notebook creates a project named `test`, creates a dummy stream, and writes and reads to it.

In [1]:
%load_ext autoreload
%autoreload 2
import asyncio
import beneath
import sys
import secrets

Create a client (assumes you have already authenticated with `beneath auth SECRET` on the command-line)

In [2]:
client = beneath.Client()

Get user and organization info

In [3]:
me = await client.admin.users.get_me()
organization_id = me["organization"]["organizationID"]

Get or create test project

In [4]:
project_name = "test"
try:
    project = await client.admin.projects.create(name=project_name, organization_id=organization_id, public=True)
except beneath.GraphQLError:
    project = await client.admin.projects.find_by_name(name=project_name)

Create test stream

In [84]:
stream_name = "dummmies"
schema = """
    type Dummmy @stream() @key(fields: "a") {
        a: String!
        b: Int!
        c: Int
        d: Bytes16
    }
"""

In [85]:
try:
    await client.admin.streams.create(schema=schema, project_id=project["projectID"])
except beneath.GraphQLError:
    await client.admin.streams.find_by_project_and_name(project_name=project_name, stream_name=stream_name)
stream = await client.find_stream(project=project_name, stream=stream_name)

Create function for generating random records for the stream

In [86]:
def generate_record():
    return {
        "a": secrets.token_urlsafe(30),
        "b": secrets.randbelow(sys.maxsize),
        "c": None,
        "d": secrets.token_bytes(16),   
    }

Write records to the stream

In [255]:
n = 1000
records = [generate_record() for _ in range(n)]
await stream.write(records)

Write records forever

In [ ]:
delay_seconds = 2
while True:
    record = generate_record()
    await stream.write(record, immediate=True)
    await asyncio.sleep(delay_seconds)

Read all records really easily

In [ ]:
df = await client.easy_read(project=project_name, stream=stream_name)
df

Read some records with lower-level APIs

In [ ]:
cursor = await stream.query_index()
df = await cursor.fetch_next(to_dataframe=True)
df

Read all the records with lower-level APIs

In [ ]:
cursor = await stream.query_index()
df = await cursor.fetch_all(to_dataframe=True)
df

Peek at the latest writes

In [ ]:
cursor = await stream.query_log(peek=True)
df = await cursor.fetch_next(to_dataframe=True)
df

Write some more and fetch the changes

In [ ]:
cursor = await stream.query_log()

n = 25
records = [generate_record() for _ in range(n)]
await stream.write(records)
await asyncio.sleep(2)

df = await cursor.fetch_next_changes(to_dataframe=True)
df

Replay values with concurrent callbacks

In [ ]:
state = { "count": 0 }
async def cb(record):
    state["count"] = state["count"] + 1
    await asyncio.sleep(0.5)

query = await stream.query_index()
await query.subscribe_replay(callback=cb)

print(f"Processed: {state['count']} records")

Helper to write records in the background for next demos

In [ ]:
async def write_forever():
    n = 200
    sleep = 1
    while True:
        records = (generate_record() for _ in range(n))
        await stream.write(records, immediate=True)
        print(f"Wrote {n} records")
        await asyncio.sleep(sleep)

Write and subscribe to new changes in parallel

In [ ]:
async def subscribe_forever():
    async def cb(record):
        print(f"Received record: {record['a']}")
        await asyncio.sleep(4)
    query = await stream.query_log()
    await query.subscribe_changes(callback=cb, max_concurrent_callbacks=2, max_prefetched_records=5)
    
await asyncio.gather(write_forever(), subscribe_forever())

Easy helper to consume history and future progress of a stream

In [ ]:
state = { "count": 0 }
async def cb(record):    
    await asyncio.sleep(0.5) # simulate network op
    
    # print progress
    state["count"] = state["count"] + 1
    if state["count"] % 1000 == 0:
        print(f"processed {state['count']} records")

await asyncio.gather(
    write_forever(),
    # THIS IS IT!
    client.easy_process_forever(project=project_name, stream=stream_name, callback=cb),
)